# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Aluno: Marcos Carvalho Ferreira

In [ ]:
!pip install pulp
import pulp

Note: you may need to restart the kernel to use updated packages.


## Problema 3

A cidade de União Paulista enfrenta uma séria carência orçamentária. Em busca de uma solução de longo prazo, a câmara de vereadores da cidade aprova uma melhoria da base da cobrança de impostos que prevê a condenação de uma área habitacional do centro da cidade e sua substituição por um conjunto habitacional moderno.

O projeto envolve duas fases:

1. demolição das casas que estão aquém do padrão para liberar terreno para o novo projeto;

2. construção do novo conjunto urbano. A seguir daremos um resumo da situação.

Um total de 300 casas aquém do padrão podem ser demolidas. Cada casa ocupa um lote de 0,25 acres. O custo de demolição de uma casa condenada é de $2.000. Os tamanhos de lotes para domicı́lios (unidades) simples, duplos, triplos e quádruplos são de 0,18; 0,28; 0,4 e 0,5 acres, respectivamente. Ruas, espaços abertos e instalações públicas ocupam 15% da
área disponı́vel.

No novo conjunto habitacional, as unidades triplas e quádruplas representam no mı́nimo 25% do total. Unidades simples devem representar no mı́nimo 20% de todas as unidades, e unidades duplas, no mı́nimo 10%. O imposto cobrado por unidade para unidades simples, duplas, triplas e quádruplas é de \$1.000, \$1.900, \$2.700 e \$3.400, respectivamente.

O custo de construção por unidade domiciliar simples, dupla, tripla e quádrupla é de \$50.000, \$70.000, \$130.000 e \$160.000, respectivamente. O financiamento acordado com um banco local será de no máximo \$15 milhões.

Quantas unidades de cada tipo devem ser construı́das para maximizar a arrecadação de impostos?

## Parâmetros

* casas máximas demolir: $C \le 300$.
* área por casa demolida: $0{,}25$ acres.
* fração para ruas/serviços: $15\%$ → fração útil $=0{,}85$.
* lotes por unidade: $l_S=0{,}18,\; l_D=0{,}28,\; l_T=0{,}40,\; l_Q=0{,}50$ (acres).
* custo demolição por casa: \$2000.
* custo construção por unidade: $c_S=50000,\; c_D=70000,\; c_T=130000,\; c_Q=160000$.
* arrecadação (imposto) por unidade: $r_S=1000,\; r_D=1900,\; r_T=2700,\; r_Q=3400$.
* financiamento máximo: \$15.000.000.

## Variáveis de decisão

* $S,D,T,Q \in \mathbb{Z}_{\ge 0}$: número de unidades **simples**, **duplas**, **triplas** e **quádruplas** a construir.
* $C \in \{0,1,\dots,300\}$: número de casas demolidas.

In [2]:
tipos = ['S', 'D', 'T', 'Q']

lote = {'S': 0.18, 'D': 0.28, 'T': 0.40, 'Q': 0.50}

imposto = {'S': 1000, 'D': 1900, 'T': 2700, 'Q': 3400}

custo_const = {'S': 50000, 'D': 70000, 'T': 130000, 'Q': 160000}

custo_demolicao_por_casa = 2000
max_casas_demolir = 300
area_por_casa = 0.25

percent_espacos_pub = 0.15

composicao = {
    'S': 0.20,
    'D': 0.10,
    'T_Q': 0.25
}

financiamento_max = 15_000_000

## Modelo matemático

### Função Objetivo:

Maximizar a arrecadação de impostos com a definição do numero de unidades construídas.

$$
\max\ f = 1000S + 1900D + 2700T + 3400Q
$$

### Restrições

1. Área disponível (área útil total = $0.85\times 0.25 C$):

$$
0.18S + 0.28D + 0.40T + 0.50Q \;\le\; 0.85\times 0.25 \; C.
$$

2. Composição mínima:

* triplas + quádruplas ≥ 25% do total:

$$
T + Q \;\ge\; 0.25\,(S+D+T+Q)
$$

* simples ≥ 20% do total:

$$
S \ge 0.20\,(S+D+T+Q)
$$

* duplas ≥ 10% do total:

$$
D \ge 0.10\,(S+D+T+Q)
$$

3. Orçamento/financiamento total máximo (demolição + construção):

$$
2000\,C + 50000S + 70000D + 130000T + 160000Q \;\le\; 15.000.000.
$$

4. Domínios:

$$
S,D,T,Q \in \mathbb{Z}_{\ge0},\quad 0 \le C \le 300,\ C\in\mathbb{Z}.
$$

### Declaração do objeto que representa o modelo matemático

In [3]:
modelo = pulp.LpProblem("PlanejamentoUrbano", pulp.LpMaximize)

### Variáveis de Decisão

In [4]:
x = pulp.LpVariable.dicts("Unidades", tipos, lowBound=0, cat='Integer')
C = pulp.LpVariable("CasasDemolidas", lowBound=0, upBound=max_casas_demolir, cat='Integer')

### Função Objetivo

In [5]:
modelo += pulp.lpSum(imposto[t] * x[t] for t in tipos), "Maximizar_Impostos"

### Restrições

In [6]:
area_util_por_casa = (1 - percent_espacos_pub) * area_por_casa
modelo += pulp.lpSum(lote[t] * x[t] for t in tipos) <= area_util_por_casa * C, "Restricao_Area"

# Expressão do total de unidades (S + D + T + Q)
U = pulp.lpSum(x[t] for t in tipos)

modelo += (x['T'] + x['Q']) >= 0.25*U, "Min25_Te_Q"
modelo += x['S'] >= 0.20*U, "Min20_S"
modelo += x['D'] >= 0.10*U, "Min10_D"

custo_total = custo_demolicao_por_casa * C + pulp.lpSum(custo_const[t] * x[t] for t in tipos)
modelo += custo_total <= financiamento_max, "Restricao_Financiamento"

In [7]:
modelo

PlanejamentoUrbano:
MAXIMIZE
1900*Unidades_D + 3400*Unidades_Q + 1000*Unidades_S + 2700*Unidades_T + 0.0
SUBJECT TO
Restricao_Area: - 0.2125 CasasDemolidas + 0.28 Unidades_D + 0.5 Unidades_Q
 + 0.18 Unidades_S + 0.4 Unidades_T <= 0

Min25_Te_Q: - 0.25 Unidades_D + 0.75 Unidades_Q - 0.25 Unidades_S
 + 0.75 Unidades_T >= 0

Min20_S: - 0.2 Unidades_D - 0.2 Unidades_Q + 0.8 Unidades_S - 0.2 Unidades_T
 >= 0

Min10_D: 0.9 Unidades_D - 0.1 Unidades_Q - 0.1 Unidades_S - 0.1 Unidades_T
 >= 0

Restricao_Financiamento: 2000 CasasDemolidas + 70000 Unidades_D
 + 160000 Unidades_Q + 50000 Unidades_S + 130000 Unidades_T <= 15000000

VARIABLES
0 <= CasasDemolidas <= 300 Integer
0 <= Unidades_D Integer
0 <= Unidades_Q Integer
0 <= Unidades_S Integer
0 <= Unidades_T Integer

### Resolvendo o problema

In [8]:
modelo.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/marcos/Área de trabalho/Mestrado/1 - Disciplina Isolada/Programação Matemática/Atividades Práticas/Atividade 1/atividade1/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/f0284f7d6ff447879f7e4d03c91f87ca-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/f0284f7d6ff447879f7e4d03c91f87ca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 47 RHS
At line 53 BOUNDS
At line 59 ENDATA
Problem MODEL has 5 rows, 5 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 343965 - 0.00 seconds
Cgl0004I processed model has 5 rows, 5 columns (5 integer (0 of which binary)) and 22 elements
Cutoff increment increased from 1e-05 to 99.9999
Cbc0012I Integer solution of -340200 found by DiveCoefficient after 0 iterations an

1

## Imprimindo as soluções do problema

In [9]:
print("Status:", pulp.LpStatus[modelo.status])
print("Arrecadação máxima (objetivo) = ${:,.2f}".format(pulp.value(modelo.objective)))
print()

total_unidades = 0
for t in tipos:
    print(f"Unidades {t}: {int(pulp.value(x[t]))}")
    total_unidades += int(pulp.value(x[t]))

print(f"\nCasas demolidas: {int(pulp.value(C))}")
print("Custo total (demolição + construção) = ${:,.2f}".format(pulp.value(custo_total)))
area_usada = sum(lote[t] * pulp.value(x[t]) for t in tipos)
area_disponivel = area_util_por_casa * pulp.value(C)
print("Área usada (acres) = {:.2f}".format(area_usada))
print("Área disponível útil (acres) = {:.2f}".format(area_disponivel))
print("Total unidades =", total_unidades)
if total_unidades>0:
    print("Percentual T+Q = {:.2%}".format((pulp.value(x['T']) + pulp.value(x['Q']))/total_unidades))
    print("Percentual S = {:.2%}".format(pulp.value(x['S'])/total_unidades))
    print("Percentual D = {:.2%}".format(pulp.value(x['D'])/total_unidades))

Status: Optimal
Arrecadação máxima (objetivo) = $343,700.00

Unidades S: 36
Unidades D: 98
Unidades T: 45
Unidades Q: 0

Casas demolidas: 245
Custo total (demolição + construção) = $15,000,000.00
Área usada (acres) = 51.92
Área disponível útil (acres) = 52.06
Total unidades = 179
Percentual T+Q = 25.14%
Percentual S = 20.11%
Percentual D = 54.75%
